## Found how to get partial_decompress=True from zarr_python library code, file "build/lib/zarr/tests/test_core.py" test function test_read_nitems_less_than_blocksize_from_multiple_chunks in line 2683

In [3]:
import zarr # Check version
zarr.__version__

'2.13.2'

Zarr Sharding pull request should show '0.0.0'

If it says e.g. 2.13.2, then you're running on the "proper" published version (without Sharding support as of 28-Sept-2022)

<font color="orange">partial_decompress works on current version 2.13, no need for Sharding</font>

### 1. I wanna test partial_decompression by accessing a random index

In [4]:
import numpy as np


numpy_arr = np.empty((65536, 16384), dtype=np.int32) # ~3GB
original_shape = numpy_arr.shape

# 50k x 150k int32 is limit of M1 Pro 32GB
for i in range(numpy_arr.shape[0]):
#     numpy_arr[i][:] = list(range(original_shape[1]*i, original_shape[1]*(i+1)))
    numpy_arr[i][:] = np.random.randint(low=0, high=original_shape[0], size=original_shape[1])

In [5]:
print("Memory used: ", numpy_arr.nbytes / 1024 / 1024 / 1024, " GB")

Memory used:  4.0  GB


In [6]:
numpy_arr[0][0:10]

array([57071, 13148, 30458, 42086, 16973, 17592, 37421, 40209, 46026,
        6304], dtype=int32)

### Following code in zarr test_core.py:2653

In [7]:
# All these extra parameters are irrelevant to _partial_decompress being enabled. Path is enough
# fsstore = zarr.storage.FSStore('./fsstore', key_separator="/", auto_mkdir=True, mode='w', normalize_keys=False)

fsstore = zarr.storage.FSStore('./fsstore4MB')

In [8]:
# chunk_size=(4096,4096)

zarr_index_list = zarr.array(numpy_arr, store=fsstore)#, chunk_store = fsstore_chunk, partial_decompress=True, chunks=chunk_size, dtype='i4')# compressor=weak_compressor, 

In [9]:
zarr_index_list.info

Type,zarr.core.Array
Data type,int32
Shape,"(65536, 16384)"
Chunk shape,"(2048, 512)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.FSStore
No. bytes,4294967296 (4.0G)
No. bytes stored,2156249432 (2.0G)
Storage ratio,2.0


In [10]:
print("_partial_decompress: ", zarr_index_list._partial_decompress)

_partial_decompress:  False


<font color="orange">As we can see, partial_decompress is False. Don't panic!</font>

In [11]:
# Flush to disk
zarr.save(fsstore, numpy_arr)

In [12]:
# Load from disk

# read_only doesn't affect partial_decompress eligibility
load_partial_decomp = zarr.Array(fsstore, read_only=True, partial_decompress=True)

In [13]:
load_partial_decomp._partial_decompress

True

#### <font color="orange">Solved! Partial_decompress=True</font>

In [14]:
load_partial_decomp.info

Type,zarr.core.Array
Data type,int32
Shape,"(65536, 16384)"
Chunk shape,"(2048, 512)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.FSStore
No. bytes,4294967296 (4.0G)
No. bytes stored,2156249432 (2.0G)
Storage ratio,2.0


In [15]:
# del numpy_arr # Don't need this anymore - save 3GB

Sometimes chunks not even initialized? Sometimes 2 arrays are written to disk (i.e. a group). Not sure why

In [16]:
load_partial_decomp[0, 100:105]

array([28904, 43120, 25319, 37625, 47725], dtype=int32)

In [17]:
load_disk_normally = zarr.open(fsstore) # , partial_decompress=True) -- This won't work !!

In [18]:
load_disk_normally._partial_decompress

False

In [19]:
load_disk_normally[0, 100:105]

array([28904, 43120, 25319, 37625, 47725], dtype=int32)

In [20]:
# Do the two match?
(load_partial_decomp[0, 100:105] == load_disk_normally[0, 100:105]).all()

True

### Let's test random access of chunks by index

In [22]:
chunk_shape = (2048, 512)

In [27]:
rand_indices_list = np.random.randint(low=0, high=chunk_shape[0], size=50)

In [28]:
%%timeit
# Read randomly from first chunk, Using partial decompression

for index in rand_indices_list:
    load_partial_decomp[0, index]

39.7 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for index in rand_indices_list:
    load_disk_normally[0, index]

58.6 ms ± 992 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### <font color="orange">Let's access the chunks using 2D accesses now</font>

In [30]:
rand_indices_tuples = (np.random.randint(low=0, high=chunk_shape[0], size=50), np.random.randint(low=0, high=chunk_shape[1], size=50))

In [31]:
rand_indices_tuples

(array([ 186, 1502,   61,  632,  467,  552,  368,  319,  120,  143, 1928,
         408, 1720, 1502,  410, 1768,  565,  948, 1486,  425,  410, 1775,
        1086, 1872,  607,  668, 1010,  513,  760, 1706, 1750,  511,  225,
        1192,  541,  440, 1609, 1014, 1511, 1754, 1532, 1560, 1793, 1120,
        1495,   39, 2027, 2015, 1902,  320]),
 array([112, 440,  34, 212, 159, 243, 334, 433, 346, 209, 277, 122,  66,
         91, 448,   0, 183,  46, 196, 446, 355, 211, 407,  98, 420, 308,
        455, 180, 386, 264, 229, 138, 453, 254, 468, 465, 237, 488, 499,
        230, 317, 309,  75, 505,  70, 437,   8, 331, 409, 329]))

In [32]:
rand_indices_tuples[0][1]

1502

TODO set range(50) programmatically

In [33]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_partial_decomp[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

37.8 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_disk_normally[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

56.2 ms ± 959 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## <font color="orange"> New experiment: access one point per chunk in the whole array</font>

In [69]:
load_partial_decomp.info

Type,zarr.core.Array
Data type,int32
Shape,"(65536, 16384)"
Chunk shape,"(2048, 512)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.FSStore
No. bytes,4294967296 (4.0G)
No. bytes stored,2156249432 (2.0G)
Storage ratio,2.0


In [75]:
32**2

1024

In [76]:
from itertools import product

In [81]:
# rand_indices_whole_array = []

xes = []
ys = []

for i in range(32): # there's 32x32 chunks. TODO get programmatically
    # 2d point access - let's not just access first row of chunk
    
    # Random point across all chunks
#     x = np.random.randint(low=i*chunk_shape[0]+5, high=(i+1)*chunk_shape[0] + 5)
#     y = np.random.randint(low=i*chunk_shape[1]+5, high=(i+1)*chunk_shape[1]+5)
    
    xes.append(i*chunk_shape[0]+50)
    ys.append(i*chunk_shape[1]+50)
    
    # "Same" point across all chunks

#     rand_indices_whole_array.append((x,y))
    
    # 1D access - makes indexing complicated
#     rand_indices_whole_array.append(np.random.randint(low=i*chunk_shape[0], high=(i+1)*chunk_shape[1]))

rand_indices_whole_array = list(product(xes, ys))
rand_indices_whole_array

[(50, 50),
 (50, 562),
 (50, 1074),
 (50, 1586),
 (50, 2098),
 (50, 2610),
 (50, 3122),
 (50, 3634),
 (50, 4146),
 (50, 4658),
 (50, 5170),
 (50, 5682),
 (50, 6194),
 (50, 6706),
 (50, 7218),
 (50, 7730),
 (50, 8242),
 (50, 8754),
 (50, 9266),
 (50, 9778),
 (50, 10290),
 (50, 10802),
 (50, 11314),
 (50, 11826),
 (50, 12338),
 (50, 12850),
 (50, 13362),
 (50, 13874),
 (50, 14386),
 (50, 14898),
 (50, 15410),
 (50, 15922),
 (2098, 50),
 (2098, 562),
 (2098, 1074),
 (2098, 1586),
 (2098, 2098),
 (2098, 2610),
 (2098, 3122),
 (2098, 3634),
 (2098, 4146),
 (2098, 4658),
 (2098, 5170),
 (2098, 5682),
 (2098, 6194),
 (2098, 6706),
 (2098, 7218),
 (2098, 7730),
 (2098, 8242),
 (2098, 8754),
 (2098, 9266),
 (2098, 9778),
 (2098, 10290),
 (2098, 10802),
 (2098, 11314),
 (2098, 11826),
 (2098, 12338),
 (2098, 12850),
 (2098, 13362),
 (2098, 13874),
 (2098, 14386),
 (2098, 14898),
 (2098, 15410),
 (2098, 15922),
 (4146, 50),
 (4146, 562),
 (4146, 1074),
 (4146, 1586),
 (4146, 2098),
 (4146, 2610),

In [82]:
len(rand_indices_whole_array) # Should equal n_chunks

1024

In [83]:
%%timeit
# Read randomly from whole array, randomly 1 point from each chunk

for tup in rand_indices_whole_array:
    load_partial_decomp[tup[0], tup[1]]

898 ms ± 69.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%%timeit
# Read randomly from whole array, randomly 1 point from each chunk
# TODO Don't know if my indexing is correct!!

for tup in rand_indices_whole_array:
    load_disk_normally[tup[0], tup[1]]

1.34 s ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [85]:
print("Speedup: ", 1.34 / .9)

Speedup:  1.488888888888889


### <font color="orange">Let's test advanced mask indexing</font>

In [86]:
# Create a 0-1 boolean mask with size same as Whole array
#    Don't know how to do just 1 chunk masking
bool_mask = np.random.randint(low=0, high=2, size=original_shape, dtype=bool)
bool_mask

array([[ True,  True, False, ...,  True,  True, False],
       [ True, False,  True, ...,  True,  True,  True],
       [ True, False, False, ...,  True,  True,  True],
       ...,
       [False,  True, False, ..., False, False,  True],
       [ True,  True,  True, ...,  True, False, False],
       [ True,  True,  True, ...,  True, False,  True]])

In [87]:
%%timeit
# Partial decompress

# _ to prevent printing
_ = load_partial_decomp.get_mask_selection(bool_mask)

53.1 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [88]:
%%timeit
# NO Partial decompress

# _ to prevent printing
_ = load_disk_normally.get_mask_selection(bool_mask)

56.3 s ± 402 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## <font color="red">Trying 16MB chunks. partial_decompress Benefits should be bigger for bigger chunks</font>

In [93]:
fsstore_8mb = zarr.storage.FSStore("./fsstore16MBChunk")

In [100]:
chunk_shape = (4096,1024) #16MB

In [94]:
zarr_index_list_16MB_Chunk = zarr.array(numpy_arr, store=fsstore_8mb, chunks = chunk_size_8mb)

In [95]:
zarr.save(fsstore_16mb, zarr_index_list_8MB_Chunk)

In [96]:
load_partial_decomp_16mb = zarr.Array(fsstore_16mb, partial_decompress=True)

In [97]:
load_disk_normally_16mb = zarr.Array(fsstore_16mb, partial_decompress=False)

In [101]:
rand_indices_tuples = (np.random.randint(low=0, high=chunk_shape[0], size=50), np.random.randint(low=0, high=chunk_shape[1], size=50))

In [102]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_partial_decomp_16mb[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

31.8 ms ± 431 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [103]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_disk_normally_16mb[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

112 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## <font color="red">64 MB chunks</font>

In [105]:
fsstore = zarr.storage.FSStore("./fsstore64MBChunk")
chunk_shape = (8192,2048)
zarr_index_list = zarr.array(numpy_arr, store=fsstore, chunks = chunk_shape)
zarr.save(fsstore, zarr_index_list)
load_partial_decomp = zarr.Array(fsstore, partial_decompress=True)
load_disk_normally = zarr.Array(fsstore, partial_decompress=False)
rand_indices_tuples = (np.random.randint(low=0, high=chunk_shape[0], size=50), np.random.randint(low=0, high=chunk_shape[1], size=50))

In [106]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_partial_decomp[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

31.5 ms ± 197 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [107]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_disk_normally[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

425 ms ± 8.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## <font color="red">256MB Chunk</font>

In [110]:
fsstore = zarr.storage.FSStore("./fsstore256MBChunk")
chunk_shape = (16384,4096)
zarr_index_list = zarr.array(numpy_arr, store=fsstore, chunks = chunk_shape)
zarr.save(fsstore, zarr_index_list)
load_partial_decomp = zarr.Array(fsstore, partial_decompress=True)
load_disk_normally = zarr.Array(fsstore, partial_decompress=False)
rand_indices_tuples = (np.random.randint(low=0, high=chunk_shape[0], size=50), np.random.randint(low=0, high=chunk_shape[1], size=50))

In [111]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_partial_decomp[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

36.7 ms ± 386 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [112]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_disk_normally[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

1.62 s ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## <font color="red">1GB Chunk</font>

In [114]:
fsstore = zarr.storage.FSStore("./fsstore1GBChunk")
chunk_shape = (32768,8192)
zarr_index_list = zarr.array(numpy_arr, store=fsstore, chunks = chunk_shape)
zarr.save(fsstore, zarr_index_list)
load_partial_decomp = zarr.Array(fsstore, partial_decompress=True)
load_disk_normally = zarr.Array(fsstore, partial_decompress=False)
rand_indices_tuples = (np.random.randint(low=0, high=chunk_shape[0], size=50), np.random.randint(low=0, high=chunk_shape[1], size=50))

In [115]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_partial_decomp[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

30.9 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [116]:
%%timeit
# Read randomly from first chunk, Not using partial decompression

for i in range(50):
    load_disk_normally[rand_indices_tuples[0][i], rand_indices_tuples[1][i]]

6.82 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [117]:
xes = []
ys = []

for i in range(2): # N chunks
    xes.append(i*chunk_shape[0]+50)
    ys.append(i*chunk_shape[1]+50)
    

rand_indices_whole_array = list(product(xes, ys))
rand_indices_whole_array

[(50, 50), (50, 8242), (32818, 50), (32818, 8242)]

In [118]:
%%timeit
# Read randomly from whole array, randomly 1 point from each chunk

for tup in rand_indices_whole_array:
    load_partial_decomp[tup[0], tup[1]]

2.46 ms ± 38.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [119]:
%%timeit
# Read randomly from whole array, randomly 1 point from each chunk

for tup in rand_indices_whole_array:
    load_disk_normally[tup[0], tup[1]]

532 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
